# SQL, a primer

## ERD Diagram

draw a ER diagram of the database and paste the screen shot here. 

you can use [this online editor for example](https://online.visual-paradigm.com/diagrams/solutions/free-erd-editor-online/)

## Query the database

### Simple Query

🤔 display the records of the table weibo_status (fetchall)

In [1]:
import sqlite3

conn = sqlite3.connect('../data/UTSEUS-shanghai-weibo-2012.db')
c = conn.cursor()

In [2]:
query = '''
SELECT 
    * 
FROM 
    weibo_status 
'''
c.execute(query)
rows = c.fetchall()
rows[:5]

[(3513796771294526,
  '吃火锅啦 我在#洪长兴羊肉馆(云南南路店)# http://t.cn/zOeOmRM',
  1153054312,
  34647,
  '2012-11-18 19:24:56+08',
  None),
 (3514061394691877,
  '我要回来啦！！！阳光明媚啊！ 我在#大连周水子国际机场# http://t.cn/zOrpZiP',
  2883949090,
  19848,
  '2012-11-19 12:56:27+08',
  None),
 (3517373581677145,
  '不要散布你的困惑和苦厄，更不要炫耀你的幸福和喜乐。那只会使它们变得廉价。做个有骨气的人，戴一副合法的表情，纵有千言万语，只与自己说。 我在#新葡京娱乐场# http://t.cn/zj5w1zd',
  2287099050,
  32214,
  '2012-11-28 16:17:55+08',
  None),
 (3518848299203707,
  'The food just so so here [围脖] 我在#首尔焖鲜汇# http://t.cn/zjtAtq9',
  1765471642,
  2214,
  '2012-12-02 17:57:53+08',
  None),
 (3518909267803148,
  '老公和同事饭局间作一打游诗:我坐2 号头，你坐2号尾， 日日思君不见君，共坐2号轨。致静安女和张江男。 我在这里：http://t.cn/zlQbJ1x',
  2709940290,
  31412,
  '2012-12-02 22:00:10+08',
  None)]

🤔 display the first 5 records of the table weibo_status (fetchone, limit)

In [3]:
query = '''
SELECT 
    * 
FROM 
    weibo_status 
LIMIT 5 
'''
c.execute(query)
rows = c.fetchall()
len(rows)

5

### query with projection

🤔 display the columns with the status text and the date from weibo_status 

In [4]:
query = '''
SELECT created_at, status
FROM weibo_status
LIMIT 1
'''
c.execute(query)
rows = c.fetchall()
rows

[('2012-11-18 19:24:56+08', '吃火锅啦 我在#洪长兴羊肉馆(云南南路店)# http://t.cn/zOeOmRM')]

🤔 display the description and gender from the table weibo_user __*renamed into `wu`*__

In [5]:
query = '''
SELECT
    wu.description, wu.gender
FROM 
    weibo_user wu
'''
c.execute(query)
rows = c.fetchall()
rows[0]

('是非浑不分，何堪辨世事', 'f')

🤔 display only the description from the table weibo_user but __rename the column `desc` instead of `description`__

In [6]:
import sqlite3

conn = sqlite3.connect('../data/UTSEUS-shanghai-weibo-2012.db')
conn.row_factory = sqlite3.Row
c = conn.cursor()

query = '''
SELECT 
    description as "desc"
FROM 
    weibo_user
'''
c.execute(query)
rows = c.fetchall()
rows[0]['desc']

'是非浑不分，何堪辨世事'

### query with selection

🤔 Select all the points of interests (POI) in jufengyuan road from the table weibo_location

In [7]:
import sqlite3

conn = sqlite3.connect('../data/UTSEUS-shanghai-weibo-2012.db')
c = conn.cursor()

In [8]:
query = '''
SELECT 
    *
FROM 
    weibo_location
WHERE 
    street = "上海市聚丰园路"
'''
c.execute(query)
rows = c.fetchall()
len(rows)

52

🤔 Select all the shops around century park: bounding box __top left__ `121.5276, 31.2256` and __bottom right__ `121.5705, 31.2097`

In [9]:
query = '''
SELECT title 
    
FROM weibo_location 
WHERE 
    longitude <= 121.5705 and 
    longitude >= 121.5276 and 
    latitude >=31.2097 and 
    latitude <=31.2256 and 
    (category_en_name like '%shop%' or category_en_name like '%store%')
'''
c.execute(query)
rows = c.fetchall()
len(rows)

18

🤔 Select all the message where the text contains `happy` 

In [10]:
query = '''
SELECT 
    status
FROM
    weibo_status
WHERE
    status like '%happy%' or status like '%快乐%'
'''
c.execute(query)
rows = c.fetchall()
len(rows)

13421

🤔 Select all the message where the text contains `happy` or `sad` 

In [11]:
query = '''
SELECT 
    status
FROM
    weibo_status
WHERE
    status like '%happy%' or 
    status like '%快乐%' or 
    status like '%sad%' or 
    status like '%悲伤%' 
'''
c.execute(query)
rows = c.fetchall()
len(rows)

15991

### query with counting

🤔 what is the most popular word in the message, `happy` or `sad`?

In [12]:
query = '''
SELECT 
    COUNT(*)
FROM
    weibo_status
WHERE
    status like '%happy%' or 
    status like '%快乐%'  
'''
c.execute(query)
happy_result = c.fetchone()[0]

query = '''
SELECT 
    COUNT(*)
FROM
    weibo_status
WHERE 
    status like '%sad%' or 
    status like '%悲伤%' 
'''
c.execute(query)
sad_result = c.fetchone()[0]

In [13]:
f'{happy_result} people are happy and {sad_result} people are sad'

'13421 people are happy and 2631 people are sad'

In [14]:
f'most people are happy! True or False: {happy_result > sad_result}'

'most people are happy! True or False: True'

### query with sorting

🤔 show the top 10 coffee place according to their distinct user checkin count 

In [15]:
query = '''
SELECT 
    title
FROM
    weibo_location
WHERE
    category_en_name = 'Café'
ORDER BY 
    distinct_user_checkin_count DESC
LIMIT 10
'''
c.execute(query)
rows = c.fetchall()
rows

[('星巴克(国贸360广场店)',),
 ('星巴克(新天地店)',),
 ('星巴克(来福士店)',),
 ('苏州中茵皇冠假日酒店',),
 ('星巴克咖啡美罗店',),
 ('古董花园 Antique Garden',),
 ('STARBUCKS COFFEE(大宁国际商业广场)',),
 ('星巴克咖啡(万达店)',),
 ('星巴克(龙之梦店)',),
 ('星巴克咖啡(南京路步行街)',)]

### query with grouping
🤔 list the locations grouped by their english category name (remove the null and unknown categories)

In [16]:
query = '''
SELECT 
    category_en_name, COUNT(*)
FROM
    weibo_location
WHERE
    category_en_name not in ("NULL", "Unknown Classification")
GROUP BY 
    category_en_name
'''
c.execute(query)
rows = c.fetchall()
rows[:3]

[('ATM', 30), ('Activity', 8), ('Adult Education', 46)]

🤔 list the top 10 locations english category names according to their count (remove the null and unknown categories)

In [17]:
query = '''
SELECT 
    category_en_name, COUNT(*) as total
FROM
    weibo_location
WHERE
    category_en_name not in ("NULL", "Unknown Classification")
GROUP BY 
    category_en_name
ORDER BY 
    total DESC
LIMIT
    10
'''
c.execute(query)
rows = c.fetchall()
rows

[('Residences', 5004),
 ('Food & Drink', 2294),
 ('Fast-food restaurant', 2254),
 ('Bus Station', 1859),
 ('Buildings', 1763),
 ('Other', 1735),
 ('Door', 1340),
 ('Live Entertainment', 1212),
 ('Shanghai cuisine', 990),
 ('Company', 811)]

🤔 list the top 10 locations english per street ordered by count. If the count is the same, order by the (category, street) having the most total checking

In [18]:
query = '''
SELECT 
    category_en_name, street, COUNT(*) as total, SUM(total_checkin_count) as checkin
FROM
    weibo_location
WHERE
    category_en_name not in ("NULL", "Unknown Classification")
GROUP BY 
    category_en_name, street
ORDER BY 
    total DESC, checkin DESC
LIMIT
    10
'''
c.execute(query)
rows = c.fetchall()
rows

[('Food & Drink', '上海市南京西路', 32, 7810),
 ('Other', '上海市南京西路', 28, 8897),
 ('Residences', '上海市场中路', 28, 2851),
 ('Buildings', '上海市延安西路', 26, 6205),
 ('Residences', '上海市斜土路', 26, 1543),
 ('Food & Drink', '上海市华山路', 25, 3606),
 ('Fast-food restaurant', '上海市共和新路', 25, 2851),
 ('Food & Drink', '上海市共和新路', 25, 1940),
 ('Airport', '上海市浦东机场路', 24, 125554),
 ('Residences', '上海市共和新路', 24, 2415)]

### query multiple tables
🤔 select the top 10 location where people got birthday party

In [19]:
query = '''
SELECT 
    title, street, COUNT(*) as total
FROM
    weibo_status ws
JOIN 
    weibo_location wl
ON
    ws.weibo_location_id = wl.weibo_location_id
WHERE
    status like '%happy birthday%' or 
    status like '%生日快乐%'
GROUP BY
    ws.weibo_location_id
ORDER BY 
    total DESC
LIMIT
    10
'''
c.execute(query)
rows = c.fetchall()
rows

[('台北纯K Party', '上海市天山路', 12),
 ('唛歌时尚KTV(淮海中路店)', '上海市淮海中路', 11),
 ('银乐迪(虹口龙之梦店)', '上海市西江湾路', 10),
 ('Richy(复兴公园店)', '上海市重庆南路', 9),
 ('上海戏剧学院(莲花路校区)', '上海市莲花路', 9),
 ('MUSE', '上海市北京东路', 7),
 ('MUSE2', '上海市兴安路', 7),
 ('银乐迪', '上海市宜山路', 6),
 ('证大·大拇指广场', '上海市芳甸路', 6),
 ('大悦城', '上海市西藏北路', 6)]

🤔 select the top 10 favorite location where the top 500 most popular people

In [20]:
query = '''
SELECT 
    title, street, category_en_name, COUNT(*) as total
FROM
    weibo_status ws
JOIN 
    weibo_location wl
ON
    ws.weibo_location_id = wl.weibo_location_id
WHERE
   ws.weibo_user_id in (
       SELECT 
           weibo_user_id
       FROM 
           weibo_user
        ORDER BY
            followers_count DESC
        LIMIT
            500
   )
GROUP BY
    ws.weibo_location_id
ORDER BY 
    total DESC
LIMIT
    10
'''
c.execute(query)
rows = c.fetchall()
rows

[('沪上一家辣肉面馆(成都北路店)', '上海市成都北路', 'Fast-food restaurant', 8),
 ('新天地', '上海市嵩山路', 'Other', 7),
 ('川杨新苑三期', '上海市孙环路', 'Residences', 7),
 ('幸福码头', '上海市中山南路', 'Residences', 7),
 ('上海明天广场JW万豪酒店', '上海市江阴路', 'Five-star hotel', 7),
 ('汇贤居(The Summit)', '上海市安福路', 'Residences', 5),
 ('镇坪路', '上海市镇坪路', 'Unknown Classification', 5),
 ('环球金融中心', '上海市东泰路', 'Unknown Classification', 5),
 ('张江科教园区', '上海市华佗路', 'Institutions of higher learning', 5),
 ('中国联通', '上海市文治路', 'Buildings', 5)]

🤔 THE FINAL QUERY
1. select the top 50 favorite location where the 500 most popular woman go 
2. select the top 50 favorite location where the 500 most popular men go
3. display the one they have in common and the one specific for each of these two genders orders by how many influencers go there

#### 1

In [21]:
query = '''
SELECT 
    title, street, category_en_name, COUNT(*) as total
FROM
    weibo_status ws
JOIN 
    weibo_location wl
ON
    ws.weibo_location_id = wl.weibo_location_id
WHERE
   ws.weibo_user_id in (
        SELECT 
            weibo_user_id
        FROM 
            weibo_user
        WHERE
            gender LIKE 'f'
        ORDER BY
            followers_count DESC
        LIMIT
            500
   )
GROUP BY
    ws.weibo_location_id
ORDER BY 
    total DESC
LIMIT
    50
'''
c.execute(query)
rows_f = c.fetchall()
rows_f

[('凤城三村', '上海市江浦路', 'Residences', 13),
 ('康泰新城', '上海市华灵路', 'Unknown Classification', 13),
 ('新天地', '上海市嵩山路', 'Other', 9),
 ('幸福码头', '上海市中山南路', 'Residences', 7),
 ('万源城朗郡', '上海市古龙路', 'Residences', 6),
 ('汤臣高尔夫别墅', '上海市龙东大道', 'Residences', 6),
 ('七韵美地苑', '上海市吴中路', 'Unknown Classification', 6),
 ('MUSE2', '上海市兴安路', 'Bar', 6),
 ('汇贤居(The Summit)', '上海市安福路', 'Residences', 5),
 ('南宜花苑', '上海市凯旋路', 'Residences', 5),
 ('镇坪路', '上海市镇坪路', 'Unknown Classification', 5),
 ('环球金融中心', '上海市东泰路', 'Unknown Classification', 5),
 ('馥邦中环广场', '上海市天山路', 'Market', 5),
 ('中国联通', '上海市文治路', 'Buildings', 5),
 ('上海外滩华尔道夫酒店(Waldorf Astoria Shanghai on the Bund)',
  '上海市延安东路',
  'Hotels, guest houses',
  5),
 ('安居兰庭', '上海市兰溪路', 'Residences', 4),
 ('muse 2', '上海市兴安路', 'Other', 4),
 ('梅赛德斯奔驰文化中心音乐俱乐部', '上海市博成路', 'Unknown Classification', 4),
 ('星巴克(东樱花苑店)', '上海市浦逸路', 'Other', 4),
 ('上海柏悦酒店', '上海市世纪大道', 'Five-star hotel', 4),
 ('复旦大学南区', '上海市政肃路', 'Institutions of higher learning', 4),
 ('上海市园林学校', '上海市杨树浦路', 'Adult Educ

#### 2

In [22]:
query = '''
SELECT 
    title, street, category_en_name, COUNT(*) as total
FROM
    weibo_status ws
JOIN 
    weibo_location wl
ON
    ws.weibo_location_id = wl.weibo_location_id
WHERE
   ws.weibo_user_id in (
        SELECT 
            weibo_user_id
        FROM 
            weibo_user
        WHERE
            gender LIKE 'm'
        ORDER BY
            followers_count DESC
        LIMIT
            500
   )
GROUP BY
    ws.weibo_location_id
ORDER BY 
    total DESC
LIMIT
    50
'''
c.execute(query)
rows_m = c.fetchall()
rows_m

[('上海财经大学中山北一路校区', '上海市新市路', 'Institutions of higher learning', 13),
 ('沪上一家辣肉面馆(成都北路店)', '上海市成都北路', 'Fast-food restaurant', 8),
 ('川杨新苑三期', '上海市孙环路', 'Residences', 7),
 ('张江科教园区', '上海市华佗路', 'Institutions of higher learning', 5),
 ('中远两湾城西区', '上海市光复西路', 'Unknown Classification', 4),
 ('和风造型私属Salon', '上海市上南路', 'Live Entertainment', 4),
 ('南新四村', '上海市下南路', 'Unknown Classification', 4),
 ('幸福路', '上海市幸福路', 'Other', 4),
 ('虹口体育馆', '上海市东体育会路', 'Food & Drink', 4),
 ('G-Star RAW旗舰店', '上海市成都北路', 'Shopping services', 4),
 ('豪享来 梅川店', '上海市梅川路', 'Western Restaurant', 4),
 ('一兆韦德(丽园店)', '上海市西藏南路', 'Unknown Classification', 4),
 ('上海明天广场JW万豪酒店', '上海市江阴路', 'Five-star hotel', 4),
 ('海伦路', '上海市四平路', 'Unknown Classification', 4),
 ('大宁国际商业广场', '上海市共和新路', 'Market', 3),
 ('夜市豆浆油条店', '上海市临潼路', 'Casual dining', 3),
 ('张江学生社区', '上海市华佗路', 'Campus Life', 3),
 ('上戏剧院', '上海市华山路', 'Concert Hall', 3),
 ('外环隧道', '上海市S20外环高速', 'Unknown Classification', 3),
 ('徐汇瑞峰酒店', '上海市肇嘉浜路', 'Four star hotels', 3),
 ('北斗星商业广场', 

#### 3

In [23]:
query = '''
SELECT 
    title, street, category_en_name, COUNT(*) as total
FROM
    weibo_status ws
JOIN 
    weibo_location wl
ON
    ws.weibo_location_id = wl.weibo_location_id
WHERE
   ws.weibo_user_id in (
        SELECT 
            weibo_user_id
        FROM 
            weibo_user
        WHERE
            gender LIKE 'f'
        ORDER BY
            followers_count DESC
        LIMIT
            500
   ) OR
    ws.weibo_user_id in (
        SELECT 
            weibo_user_id
        FROM 
            weibo_user
        WHERE
            gender LIKE 'm'
        ORDER BY
            followers_count DESC
        LIMIT
            500
   )
GROUP BY
    ws.weibo_location_id
ORDER BY 
    total DESC
LIMIT
    50
'''
c.execute(query)
rows_a = c.fetchall()
rows_a

[('凤城三村', '上海市江浦路', 'Residences', 13),
 ('上海财经大学中山北一路校区', '上海市新市路', 'Institutions of higher learning', 13),
 ('康泰新城', '上海市华灵路', 'Unknown Classification', 13),
 ('新天地', '上海市嵩山路', 'Other', 10),
 ('沪上一家辣肉面馆(成都北路店)', '上海市成都北路', 'Fast-food restaurant', 8),
 ('川杨新苑三期', '上海市孙环路', 'Residences', 7),
 ('幸福码头', '上海市中山南路', 'Residences', 7),
 ('MUSE2', '上海市兴安路', 'Bar', 7),
 ('上海明天广场JW万豪酒店', '上海市江阴路', 'Five-star hotel', 7),
 ('万源城朗郡', '上海市古龙路', 'Residences', 6),
 ('汤臣高尔夫别墅', '上海市龙东大道', 'Residences', 6),
 ('上戏剧院', '上海市华山路', 'Concert Hall', 6),
 ('七韵美地苑', '上海市吴中路', 'Unknown Classification', 6),
 ('汇贤居(The Summit)', '上海市安福路', 'Residences', 5),
 ('南宜花苑', '上海市凯旋路', 'Residences', 5),
 ('镇坪路', '上海市镇坪路', 'Unknown Classification', 5),
 ('环球金融中心', '上海市东泰路', 'Unknown Classification', 5),
 ('张江科教园区', '上海市华佗路', 'Institutions of higher learning', 5),
 ('馥邦中环广场', '上海市天山路', 'Market', 5),
 ('中国联通', '上海市文治路', 'Buildings', 5),
 ('上海外滩华尔道夫酒店(Waldorf Astoria Shanghai on the Bund)',
  '上海市延安东路',
  'Hotels, guest houses',

In [24]:
for i in range(50):
    for j in range(50):
        if rows_f[i][0]==rows_a[j][0]:
            if rows_f[i][3]==rows_a[j][3]:
                print(rows_f[i],'FEMALE SPECIFIC')
                break
        if rows_m[i][0]==rows_a[j][0]:
            if rows_m[i][3]==rows_a[j][3]:
                print(rows_m[i],'MALE SPECIFIC')
                break
        if rows_f[i][0]==rows_a[j][0]:
            if rows_f[i][3]!=rows_a[j][3]:
                print(rows_f[i],'COMMON PLACES')
                break

('凤城三村', '上海市江浦路', 'Residences', 13) FEMALE SPECIFIC
('康泰新城', '上海市华灵路', 'Unknown Classification', 13) FEMALE SPECIFIC
('新天地', '上海市嵩山路', 'Other', 9) COMMON PLACES
('幸福码头', '上海市中山南路', 'Residences', 7) FEMALE SPECIFIC
('万源城朗郡', '上海市古龙路', 'Residences', 6) FEMALE SPECIFIC
('汤臣高尔夫别墅', '上海市龙东大道', 'Residences', 6) FEMALE SPECIFIC
('七韵美地苑', '上海市吴中路', 'Unknown Classification', 6) FEMALE SPECIFIC
('MUSE2', '上海市兴安路', 'Bar', 6) COMMON PLACES
('汇贤居(The Summit)', '上海市安福路', 'Residences', 5) FEMALE SPECIFIC
('南宜花苑', '上海市凯旋路', 'Residences', 5) FEMALE SPECIFIC
('镇坪路', '上海市镇坪路', 'Unknown Classification', 5) FEMALE SPECIFIC
('环球金融中心', '上海市东泰路', 'Unknown Classification', 5) FEMALE SPECIFIC
('馥邦中环广场', '上海市天山路', 'Market', 5) FEMALE SPECIFIC
('中国联通', '上海市文治路', 'Buildings', 5) FEMALE SPECIFIC
('上海外滩华尔道夫酒店(Waldorf Astoria Shanghai on the Bund)', '上海市延安东路', 'Hotels, guest houses', 5) FEMALE SPECIFIC
('安居兰庭', '上海市兰溪路', 'Residences', 4) FEMALE SPECIFIC
('muse 2', '上海市兴安路', 'Other', 4) FEMALE SPECIFIC
('梅赛德斯奔驰文化中心音乐